# 10. 기울기 소실(Gradient Vanishing)과 폭주(Exploding)
---
깊은 신경망을 학습하다보면 역전파 과정에서 입력층으로 갈 수록 기울기가 점차적으로 작아지는 현상이 발생. 입력층에 가까운 층들에서 가중치들이 업데이트가 제대로 되지 않으면 결국 최적의 모델을 찾을 수 없게 된다. 이를 기울기 소실이라 함.  
반대의 경우로 기울기가 점차 커지더니 가중치들이 비정상적으로 큰 값이 되면서 발산하기도 한다. 이를 기울기 폭주라 함.

---
### 1. ReLU와 ReLU의 변형들
시그로이드 함수를 사용하면 입력의 절대닶이 클 경우에 시그모이드 함수의 출력값이 0또는 1에 수렴하면서 기울기가 0에 가까워진다.  
그래서 역전파 과정에서 전파 시킬 기울기가 점차 사라져 입력층 방향으로 갈 수록 제대로 역전파가 되지 않는 기울기 소실 문제가 발생할 수 있다.  

기울기 소실을 완화하는 간단한 방법은 윽닉층 활성화 함수로 시그모이드나 하이퍼볼릭탄젠츠 함수 대신에 ReLU나 ReLU의 변형 함수와 같은 Leaky ReLU를 사용하는 것이다. 

---
### 2. 가중치 초기화(Weight initialization)
같은 모델을 훈련시키더라도 가중치가 초기에 어떤 값을 가졌느냐에 따라 모델의 훈련 결과가 달라지기도 한다.  가중치 초기화만 적절히 해줘도 기울기 소실과 같은 문제를 방지할 수 있다.  

1. 세이비어 초기화(Xavier Initialization)
   균등 분포, 정규 분포로 초기화 할 때 두가지 경우로 나뉜다.  
   ![distribution](../img/distribution.png)
   이전 층의 뉴런 개수($n_{in}$)와 다음 층의 뉴런 개수($n_{out}$)를 가지고 식을 세운다. 
   
    
   균등 분포를 사용하여 가중치를 초기화할 경우 다음과 같은 균등 분포 범위를 사용하라고 논문에서는 말한다.  
   $W \sim Uniform(-\sqrt{\frac{6}{ {n}_{in} + {n}_{out} }}, +\sqrt{\frac{6}{ {n}_{in} + {n}_{out} }})$  
   다시 말해 $\sqrt{\frac{6}{ {n}_{in} + {n}_{out} }}$ 를 m이라고 할 때, -m과 +m사이의 균등 분포를 의미한다.  

   정규 분포를 사용하여 가중치를 초기화할 경우에는 평균이 0이고 표준 편차 σ가 다음을 만족하도록 한다.  
   $σ=\sqrt{\frac { 2 }{ { n }_{ in }+{ n }_{ out } } }$


   세이비어 초기화는 여러층의 기울기 분산 사이에 균형을 맞춰서 특정 층이 너무 주목을 받거나 다른 층이 뒤쳐지는 것을 막는다.  
   시그모이드, 하이퍼볼릭 탄젠트와 같은 S가 형태인 활성화 함수와 함께 사용할 경우 좋은 성능을 보이지만, ReLU와 함께 사용할 경우는 성능이 좋지 않다.  
   ReLU함수 또는 ReLU 변형 함수들을 활성화 함수로 사용할 경우에는 다른 초기화 방법을 사용하는 것이 좋은데, 이를 He 초기화라고 한다.  
--  
1. He 초기화(He initialization)
   He 초기화는 세이비어 초기화와 유사하게 정규 분포와 균등 분포 두 가지 경우로 나뉜다.
   다만 He 초기화는 세이비어와는 다르게 다음 층의 뉴런의 수를 반영하지 않는다.  

   He 초기화는 균등 분포로 초기화 할 경우 다음과 같은 균등 분포 범위를 갖도록 한다.  
   $W\sim Uniform(- \sqrt{\frac { 6 }{ { n }_{ in } } } , \space\space + \sqrt{\frac { 6 }{ { n }_{ in } } } )$


   정규 분포로 초기화할 경우에는 표준 편차 σ가 다음을 만족하도록 한다.
   $σ=\sqrt{\frac { 2 }{ { n }_{ in } } }$


   ReLU + He 초기화 방법이 좀 더 보편적이다.  

---
### 3. 배치 정규화(Batch Normalization)
위의 가중치 초기화 방법으로 기울기 소실과 폭주를 완화시킬 수 있지만, 훈련 중에 언제든 다시 발생할 수 있다.   
또 다른 방법으로 배치 정규화(Batch Normalization)이 있다.  
배치 정규화는 인공 신경망의 각 층에 들어가는 입력을 평균과 분산으로 정규화하여 학습을 효율적으로 만든다.  


1. 내부 공변량 변화(Internal Covariate Shift)
배치 정규화를 이해하기 위해서 우선 내부 공변량 변화를 이해해야 한다.  
내부 공변량 변화: 학습 과정에서 층 별로 입력 데이터의 분포가 달라지는 현상  
이전 층들의 학습에 의해 이전 층의 가중치 값이 바뀌게 되면, 현재 층에 전달되는 입력 데이터의 분포가 현재 층이 학습했던 시점의 분포와 차이가 발생한다.  
배치 정규화를 제안한 논문에서는 기울기 소실, 폭주등의 불안정성이 층마다 입력의 분포가 달라지기 때문에 발생한다고 주장한다.  


2. 배치 정규화(Batch Normalization)
배치 정규화는 한 번에 들어오는 배치 단위로 정규화 하는 것을 말한다.  
배치 정규화는 각 층에서 활성화 함수를 통과하기 전에 수행 된다.  

**배치 정규화 과정 요약**
입력에 대해 평균을 0으로 만들고, 정규화를 한다.  
정규화 된 데이터에 대해서 스케일과 시프트를 수행한다.  
이때 두 개의 매개변수 γ와 β를 사용하는데, γ는 스케일을 위해 사용하고 β는 시프트에 사용되며 다음 레이어에 일정한 범위의 값들만 전달되게 한다.  

**배치 정규화의 수식**  
BN은 배치 정규화를 의미한다.  
input: 미니 배치 $B = \{{x}^{(1)}, {x}^{(2)}, ..., {x}^{(m)}\}$  
output: $y^{(i)} = BN_{γ, β}(x^{(i)})$  
![bn_01](../img/bn_01.JPG)  

배치 정규화는 학습 시 배치 단위의 평균과 분산들을 차례대로 받아 이동 평균과 이동 분산을 저장해놓았다가 테스트 할 때는 해당 배치의 평균과 분산을 구하지 않고 구해놓았던 평균과 분산으로 정규화를 진행한다.  

- BN을 사용하면 sigmoid나 tanh을 사용하더라도 기울기 소실 문제가 크게 개선된다.
- 가중치 초기화에 훨씬 덜 민감해짐.
- 훨씬 큰 학습률을 사용할 수 있어 학습 속도를 개선함.
- 미니 배치마다 평균과 표준편차를 계산하므로 훈련 데이터에 일종의 잡음을 넣는 부수 효과도 과적합을 방지하는 효과까지 핸다. 하지만 부수적인 효과이므로 드롭아웃과 함께 사용하는 것이 좋다.
- BN는 모델을 복잡하게 하며, 추가 계산을 하는 것이므로 테스트 데이터에 대한 예측 시에 실행 시간이 느려진다. 서비스 속도를 고려하는 관점에서는 고민해보고 사용할 것.


3. 배치 정규화의 한계
   1. 미니 배치 크기에 의존적
      너무 작은 배치 크기에서는 잘 동작하지 않을 수 있다.  
      단적으로 배치 크기를 1로 하게 되면 분산은 0이 된다.  
      작은 미니 배치에서는 배치 정규화의 효과가 극단적으로 작용되어 훈련에 악영향을 줄 수 있다.  
      BN을 사용할 때는 작은 미니 배치보다 크기가 어느정도 되는 미니 배치에서 하는 것이 좋다.  


   2. RNN에 적용하기 어려움
      RNN는 각 시점(time step)마다 다른 통계치를 가진다. 이는 RNN에 배치 정규화를 적용하기 어렵게 만든다.  
      RNN에 BN을 적용하는 몇가지 방법이 제시 되었지만 그보다 층 정규화(layer normalization)을 소개한다.  


---
### 4. 층 정규화(Layer Normalization)

![bn_02](../img/bn_02.png)  
[참고 블로그](https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/)  
Batch Normalization(이하 BN)과 Layer Normalization(이하 LN) 모두 값들이 심하게 차이나는 정도를 줄이기 위해서 사용하는데 그 방향이 서로 다르다.  

- BN: **각 feature의 평균과 분산**을 구해서 batch에 있는 **각 feature 를 정규화**
- LN: **각 input의 feature들에 대한 평균과 분산**을 구해서 batch에 있는 **각 input을 정규화**

따라서 BN은 batch size와 관련이 깊지만 LN은 각 input에 대해서만 처리되므로 batch size와는 상관이 없다.  